# Create a dataset of non-digits from MNIST

Can we create a dataset of non-digits from MNIST that can be used by mnist-or-not-training.ipynb?

This is still very much a work in progress - even so, I'm not expecting this approach give us great results ...

|                                            | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | notes                 |
|--------------------------------------------|---|---|---|---|---|---|---|---|---|---|-----------------------|
| zero 1 square at random (1 quarter size)   | y |   | y | y | y | y | y | y | y | y |                       |
| zero 1 quadrant                            | y |   | y | y | y | y | y | y | y | y |                       |
| zero 1 half vertical                       | y | y | y | y | y | y | y | y | y | y |                       |
| zero 1 half horizontally                   | y |   | y | y | y | y | y | y | y | y |                       |
| flip top half vertically                   | y |   | y | y | y | y | y | y |   |   |                       |
| flip vertial                               |   |   | y |   | y | y | y | y |   | y |                       |
| flip horizontal                            |   |   | y | y | y | y | y | y |   | y |                       |
| overlay a different number                 | y | y | y | y | y | y | y | y | y | y | 8 and 6 might make 8! |
| overlay same digit rotated 90, 270         |   | y | y | y | y | y | y | y | y | y |                       |
| overlay same digit rotated 180             |   |   | y |   | y | y |   | y |   |   |                       |
| random scribble                            | y | y | y | y | y | y | y | y | y | y |                       |
| half of 1 number joined to half of another |   |   |   |   |   |   |   |   |   |   |                       |
| Rotate: 90, 270                            |   | y | y | y | y | y | y | y | y | y |                       |
| Rotate: 180                                |   |   | y | y | y | y |   | y |   |   |                       |
| smiley face                                | y |   |   |   |   |   |   |   |   |   |                       |

See: https://thisdavej.com/copy-table-in-excel-and-paste-as-a-markdown-table/

In [1]:
import torch
from torchvision.transforms import functional as F
from PIL import Image
from fastai.datasets import untar_data,URLs
from random import randint
from functools import partial
from random import choice,randint
import inspect

In [2]:
path = untar_data(URLs.MNIST); path

PosixPath('/home/peter/.fastai/data/mnist_png')

In [3]:
img0_path=path/'training/0/210.png'
img0=Image.open(img0_path)
img4_path=path/'training/4/354.png'
img4=Image.open(img4_path)
img4

In [4]:
def open_random(input_path, kind, digits):
    "pick an image at random that is of the right `kind` and is one of `digits`"
    path=input_path/f'{kind}/{choice(digits)}'
    return Image.open(choice(list(path.iterdir())))

In [5]:
open_random(path, 'training', [0,1,2,3])

In [6]:
img0.rotate(90)

In [7]:
F.vflip(img4)

In [8]:
F.hflip(img4)

In [9]:
def rotate(img,angle=90):
    return img.rotate(angle)

In [10]:
rotate(img4, 270) # F.rotate -> error )o:

In [11]:
rotate_180=partial(rotate,angle=180)
rotate_270=partial(rotate,angle=270)

In [12]:
rotate_180(img4)

In [13]:
def erase_quadrant(img,i=0,j=0,v=0):
    return F.to_pil_image(F.erase(F.to_tensor(img),i,j,14,14,v))

In [14]:
erase_quadrant(img4)

In [15]:
erase_quadrant(img4,14,14)

In [16]:
erase_quadrant(img4.rotate(45),0,0,.75).rotate(-45)

In [37]:
def erase_random_quadrant(img):
    i,j=randint(0,1)*14,randint(0,1)*14
    return F.to_pil_image(F.erase(F.to_tensor(img),i,j,14,14,0))

In [76]:
erase_random_quadrant(img0)

In [17]:
def erase_random_square(img):
    i,j=randint(0,14),randint(0,14)
    return F.to_pil_image(F.erase(F.to_tensor(img),i,j,14,14,0))

In [18]:
erase_random_square(img4)

In [19]:
def overlay_rotated(img, img2=None, angle=90):
    a=F.to_tensor(img)
    b=F.to_tensor((img if img2 is None else img2).rotate(angle))
    return F.to_pil_image(torch.clamp(a+b,0,1))

In [20]:
overlay_rotated(img4,img0,0)

In [21]:
overlay_rotated(img4)

In [86]:
def overlay_with_random(img,input_path,kind,digit):
    digits=[i for i in range(10) if i!=digit]
    img2=open_random(input_path,kind,digits)
    return overlay_rotated(img,img2,randint(0,359))

In [115]:
overlay_with_random(img0,path,'training',0)

In [22]:
def hsplice(img1,img2):
    a=F.to_tensor(img1)[:,0:14,0:28]
    b=F.to_tensor(img2)[:,15:28,0:28]
    return F.to_pil_image(torch.cat([a,b],1))

In [23]:
hsplice(img0,img4)

In [24]:
hsplice(img4,img0)

In [25]:
def hsplice_with_random(img,input_path,kind,digit):
    digits=[i for i in range(10) if i!=digit]
    imgs=[img, open_random(input_path,kind,digits)]
    if randint(0,1): imgs.reverse()
    return hsplice(*imgs)

In [26]:
hsplice_with_random(img0,path,'training',0)

In [28]:
def flip_half(img,part='top'): # top, bottom, both
    t=F.to_tensor(img)
    a=t[:,0:14,0:28]
    b=t[:,15:28,0:28]
    if part in ['top','both']: a=a.flip([1])
    if part in ['bottom','both']: b=b.flip([1])
    return F.to_pil_image(torch.cat([a,b],1))

In [29]:
flip_half(img4, 'both')

In [119]:
def flip_half_random(img):
    return flip_half(img,choice(['top','bottom','both']))

In [130]:
flip_half_random(img0)

## Start by copying MNIST to a new `mnist_or_not` folder

In [176]:
from pathlib import Path
def cp_mnist(kind):
    p = path/kind
    o = output_path = path.parent/f'mnist_or_not/{kind}'
    print(f'copying from {p} to {o}')
    !cp -r $p/. $o

cp_mnist('training')
cp_mnist('testing')

copying from /home/peter/.fastai/data/mnist_png/training to /home/peter/.fastai/data/mnist_or_not/training
copying from /home/peter/.fastai/data/mnist_png/testing to /home/peter/.fastai/data/mnist_or_not/testing


## Read MNIST images, convert them to not be digits, and save them to the new `mnist_or_not` folder

In [30]:
def can_call_with_n_positional_args(f, n):
    "return true if f can be called with n positional arguments, false otherwise"
    def _len(l): return 0 if l is None else len(l)
    fas = inspect.getfullargspec(f)
    def _min(): return _len(fas.args) - _len(fas.defaults)
    def _max(): return 99999 if fas.varargs else len(fas.args)
    if inspect.ismethod(f): n += 1 # add one for self
    return n >= _min() and n <= _max()

In [160]:
def _create_not_digits(input_path, output_path, kind, digit, converters):
     print('converting', kind, digit)
    (output_path/f'{kind}/not{digit}').mkdir(parents=True, exist_ok=True)
    for i, f in enumerate((input_path/f'{kind}/{digit}').iterdir()):
        converter=converters[i % len(converters)]
        output_file=output_path/f'{kind}/not{digit}/{f.name}'
        args=[Image.open(f)]
        if can_call_with_n_positional_args(converter,4): args=args+[input_path,kind,digit]
        converter(*args).save(output_file)
        
def create_not_digits(path, digit, converters):
    output_path = path.parent/'mnist_or_not'
    _create_not_digits(path, output_path, 'training', digit, converters)
    _create_not_digits(path, output_path, 'testing', digit, converters)

In [177]:
base_converters=[erase_random_square,hsplice_with_random,erase_random_quadrant,overlay_with_random,overlay_with_random]
all_converters=base_converters+[rotate,rotate_180,rotate_270,overlay_rotated,F.vflip,F.hflip]
digits_and_converters={
    0:base_converters+[flip_half_random],
    1:base_converters+[rotate,rotate_270,overlay_rotated],
    2:all_converters,
    3:base_converters+[rotate,rotate_270,overlay_rotated,F.hflip],
    4:all_converters,
    5:all_converters,
    6:base_converters+[rotate,rotate_270,overlay_rotated,F.hflip],
    7:all_converters,
    8:base_converters+[rotate,rotate_270,overlay_rotated],
    9:base_converters+[rotate,rotate_270,overlay_rotated,F.hflip],
}
for digit,converters in digits_and_converters.items():
    create_not_digits(path,digit,converters)

## TODO: fix this so we can convert faster

In [155]:
# from concurrent.futures import ProcessPoolExecutor
# params=[(path,digit,converters) for digit,converters in digits_and_converters.items()]
# with ProcessPoolExecutor() as executor:
#     executor.map(create_not_digits, params)